# Define functions to create feature matrix for the signatures
# 1. Removal of Noises in an image - Contrast, Brightness, Binary, Otsu threshold & Thinning
# 2. Create functions for creating features - Area covered by the signature, Centre of Gravity, Aspect Ratio, Connected Components, etc. 
# Total Features = 19 

In [10]:
import numpy as numpy
from matplotlib import pyplot as matplot
%matplotlib inline
import pandas as pandas
import math
from sklearn import preprocessing
from sklearn import svm
import cv2
import os
import ntpath
from sklearn.metrics.pairwise import cosine_similarity
import fnmatch

data = []
Filename = []
Person = []
Testdata = []
TestFilename = []
TestPerson = []


def crop_image(img,tol=0):
    mask = img>tol
    return img[numpy.ix_(mask.any(1),mask.any(0))]
def thinning(img):
    #img = cropimage
    size = numpy.size(img)
    skel = numpy.zeros(img.shape,numpy.uint8)

    ret,img = cv2.threshold(img,127,255,0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    done = False

    while( not done):
        eroded = cv2.erode(img,element)
        temp = cv2.dilate(eroded,element)
        temp = cv2.subtract(img,temp)
        skel = cv2.bitwise_or(skel,temp)
        img = eroded.copy()

        zeros = size - cv2.countNonZero(img)
        if zeros==size:
            done = True

  #  plt.imshow(skel,'gray')
   # plt.show()
    return skel
def coords(thinnedimage):
    rows,cols=thinnedimage.shape
    img_topleft=thinnedimage[0:int(rows/2),0:int(cols/2)]
    img_topright=thinnedimage[int(rows/2)+1:rows,0:int(cols/2)]
    img_bottomleft=thinnedimage[0:int(rows/2),int(cols/2)+1:cols]
    img_bottomright=thinnedimage[int(rows/2)+1:rows,int(cols/2)+1:cols]
    
    topleft_x,topleft_y=COG(img_topleft)
    topright_x,topright_y=COG(img_topright)
    bottomleft_x,bottomleft_y=COG(img_bottomleft)
    bottomright_x,bottomright_y=COG(img_bottomright)

    return topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y

def COG(img):
    x_cor=0
    xrun_sum=0
    y_cor=0
    yrun_sum=0
    #print(img.shape)
    for i in range(img.shape[0]):
        x_cor+=sum(img[i])*i/255
        xrun_sum+=sum(img[i])/255

    for i in range(img.shape[1]):
        y_cor+=sum(img[:,i])*i/255
        yrun_sum+=sum(img[:,i])/255
        #print(img.shape[1]) 
        if yrun_sum==0:
            x_pos=0
        else:
            x_pos=y_cor/(yrun_sum)
        if xrun_sum==0:
            y_pos=0
        else:
            y_pos=x_cor/(xrun_sum)
        
   # print(x_pos)
  #  print(y_pos)
    
    return (x_pos/img.shape[1],y_pos/img.shape[0])

def tan_i(x):
    #print(x)
    if x[0]==0:
        return 90
    return math.degrees(math.atan(x[1]/x[0]))

def tan(thinnedimage):
    rows,cols=thinnedimage.shape
    
    img_tl1=thinnedimage[0:int(rows/2),0:int(cols/4)]
    img_tl2=thinnedimage[0:int(rows/2),int(cols/4)+1:int(cols/2)]
    
    img_tr1=thinnedimage[0:int(rows/2),int(cols/2)+1:int(0.75*cols)]
    img_tr2=thinnedimage[0:int(rows/2),int(0.75*cols)+1:cols]
    
    img_bl1=thinnedimage[int(rows/2)+1:rows,0:int(cols/4)]
    img_bl2=thinnedimage[int(rows/2)+1:rows,int(cols/4)+1:int(cols/2)]
    
    img_br1=thinnedimage[int(rows/2)+1:rows,int(cols/2)+1:int(0.75*cols)]
    img_br2=thinnedimage[int(rows/2)+1:rows,int(0.75*cols)+1:cols]
    

    #plt.imshow(timg,'gray')
    #plt.show()
    
    tl1=tan_i(COG(img_tl1))
    tl2=tan_i(COG(img_tl2))
    tr1=tan_i(COG(img_tr1))
    tr2=tan_i(COG(img_tr2))
    bl1=tan_i(COG(img_bl1))
    bl2=tan_i(COG(img_bl2))
    br1=tan_i(COG(img_br1))
    br2=tan_i(COG(img_br2))
    
    #plt.imshow(img_br1,'gray')
    #plt.show()
    #print(COG(img_br1))
    return tl1,tl2,tr1,tr2,bl1,bl2,br1,br2

def createfingerprint_Genuine(image):
    aspect_ratio = 1.0*(image.shape[1]/image.shape[0])
    ret,thresh1 = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
    ret,thresh2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
    ret,thresh3 = cv2.threshold(image,127,255,cv2.THRESH_TRUNC)
    ret,thresh4 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO)
    ret,thresh5 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO_INV)
    blur = cv2.GaussianBlur(thresh1,(5,5),0)
    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image=numpy.invert(th3)
    
    cropimage=crop_image(image,tol=0)
    area=cv2.countNonZero(cropimage)
    area=cv2.countNonZero(cropimage)/(cropimage.shape[0]*cropimage.shape[1])

    img1=numpy.invert(cropimage)
    connectcomp=cv2.connectedComponents(img1)[0]

    thinnedimage=thinning(cropimage)
    #Thinning the image!

    topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y=coords(thinnedimage)
    tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y = tan(thinnedimage)
    
        #Extracting features

    a=pandas.Series([area,connectcomp,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y,aspect_ratio],index=
                    ["Norm_area","connected_comps","topleft_x","topleft_y","topright_x","topright_y","bottomleft_x","bottomleft_y","bottomright_x","bottomright_y","tan_topleft_x", "tan_topleft_y", "tan_topright_x", "tan_topright_y", "tan_bottomleft_x", "tan_bottomleft_y", "tan_bottomright_x", "tan_bottomright_y","Aspect_Ratio"])
    data.append(a)

def createfingerprint_Test(image):
    aspect_ratio = 1.0*(image.shape[1]/image.shape[0])
    ret,thresh1 = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
    ret,thresh2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
    ret,thresh3 = cv2.threshold(image,127,255,cv2.THRESH_TRUNC)
    ret,thresh4 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO)
    ret,thresh5 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO_INV)
    blur = cv2.GaussianBlur(thresh1,(5,5),0)
    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image=numpy.invert(th3)
    
    cropimage=crop_image(image,tol=0)
    area=cv2.countNonZero(cropimage)
    area=cv2.countNonZero(cropimage)/(cropimage.shape[0]*cropimage.shape[1])

    img1=numpy.invert(cropimage)
    connectcomp=cv2.connectedComponents(img1)[0]

    thinnedimage=thinning(cropimage)
    #Thinning the image!

    topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y=coords(thinnedimage)
    tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y = tan(thinnedimage)
    
        #Extracting features

    b=pandas.Series([area,connectcomp,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y,aspect_ratio],index=
                    ["Norm_area","connected_comps","topleft_x","topleft_y","topright_x","topright_y","bottomleft_x","bottomleft_y","bottomright_x","bottomright_y","tan_topleft_x", "tan_topleft_y", "tan_topright_x", "tan_topright_y", "tan_bottomleft_x", "tan_bottomleft_y", "tan_bottomright_x", "tan_bottomright_y","Aspect_Ratio"])
    Testdata.append(b)    
    

def create_training_Genuine_dataset(m):
    os.chdir(Genuinedir)
    filecount = len(fnmatch.filter(os.listdir(), '*.png'))
    n = int(filecount/m)
    for j in range(1,m+1):
        for i in range(1,n+1):
            path = f"C:\\Siddhartha\\Confidential Documents\\Stretch Projects\\Data Science\\Signature Verification\\Train_Genuine\\00{j}_{i}.png"
            fn = ntpath.basename(path)
            Filename.append(fn)
            tn = j
            Person.append(tn)
            image = cv2.imread(path,0)
            createfingerprint_Genuine(image)
            Original_Signature_Table=pandas.DataFrame(data)
            Original_Cosine = pandas.DataFrame(cosine_similarity(Original_Signature_Table), index = None)
            
    Original_Signature_Table['Filename'] = Filename
    Original_Cosine['Filename'] = Filename
    Trainingdatacol = Original_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Original_Signature_Table = Original_Signature_Table[Trainingdatacol]
    Original_Signature_Table['Person'] = Person
    Trainingdatacol = Original_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Original_Signature_Table = Original_Signature_Table[Trainingdatacol]
    Original_Signature_Table['Category'] = "Genuine"
    Cosinecolumns = Original_Cosine.columns.tolist()
    Cosinecolumns = Cosinecolumns[-1:]+Cosinecolumns[:-1]
    Original_Cosine = Original_Cosine[Cosinecolumns]
    #Original_Signature_Table = pandas.concat([Original_Signature_Table,Cosine],axis = 1)
    Original_Signature_Table.to_csv('Training.csv', index = False)
    Original_Cosine.to_csv('Cosine_Training.csv', index = False)
    Summary_Table = pandas.DataFrame(Original_Signature_Table.describe()).T
    Summary_Table.to_csv('Training_Summary.csv', index = False)
    Original_Signature_Table
    Original_Cosine
    print('Preprocessing of Training Data Completed')
    
def create_test_dataset(x):
    os.chdir(Testdir)
    Testcount = len(fnmatch.filter(os.listdir(), '*.png'))
    y = int(Testcount/x)
    for j in range(1,x+1):
        for i in range(1,y+1):
            path = f"C:\\Siddhartha\\Confidential Documents\\Stretch Projects\\Data Science\\Signature Verification\\Test\\00{j}_{i}.png"
            fn = ntpath.basename(path)
            tn = j
            TestFilename.append(fn)
            TestPerson.append(tn)
            image = cv2.imread(path,0)
            createfingerprint_Test(image)
            Test_Signature_Table=pandas.DataFrame(Testdata)
            Test_Cosine = pandas.DataFrame(cosine_similarity(Test_Signature_Table), index = None)
            
            

    Test_Signature_Table['Filename'] = TestFilename
    Test_Cosine['Filename'] = TestFilename
    Trainingdatacol = Test_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Test_Signature_Table = Test_Signature_Table[Trainingdatacol]
    Test_Signature_Table['Person'] = TestPerson
    Trainingdatacol = Test_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Test_Signature_Table = Test_Signature_Table[Trainingdatacol]
    Test_Signature_Table['Category'] = "Test"
    Cosinecolumns = Test_Cosine.columns.tolist()
    Cosinecolumns = Cosinecolumns[-1:]+Cosinecolumns[:-1]
    Test_Cosine = Test_Cosine[Cosinecolumns]
    #Original_Signature_Table = pandas.concat([Original_Signature_Table,Cosine],axis = 1)
    Test_Signature_Table.to_csv('Test.csv', index = False)
    Test_Cosine.to_csv('Cosine_Test.csv', index = False)
    Summary_Table = pandas.DataFrame(Test_Signature_Table.describe()).T
    Summary_Table.to_csv('Test_Summary.csv', index = False)
    Test_Signature_Table
    Test_Cosine
    print('Preprocessing of Test Data Completed')

# Create Original DataSet

In [6]:
Genuinedir = f'C:\\Siddhartha\\Confidential Documents\\Stretch Projects\\Data Science\\Signature Verification\\Train_Genuine'
Fileformat = "\\00{j}_{i}.png"
create_training_Genuine_dataset(10)

Preprocessing of Training Data Completed


In [125]:
os.chdir(Genuinedir)
Original_Signature_Table = pandas.read_csv("Training.csv")
Cosine_Training = pandas.read_csv("Cosine_Training.csv")

In [126]:
Mean_Original_Signature = pandas.DataFrame(Original_Signature_Table.groupby('Person').mean().reset_index())

In [128]:
Mean_Original_Signature

,Person,Norm_area,connected_comps,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x,tan_topleft_y,tan_topright_x,tan_topright_y,tan_bottomleft_x,tan_bottomleft_y,tan_bottomright_x,tan_bottomright_y,Aspect_Ratio
0,1,0.027105,2.833333,0.690186,0.478752,0.407682,0.577215,0.540331,0.564867,0.288091,0.244816,40.648912,45.046052,49.203607,42.799150,59.999457,38.776304,34.618292,41.829599,2.259635
1,2,0.066962,2.083333,0.458547,0.587666,0.495477,0.187518,0.600000,0.759770,0.470382,0.262006,48.472970,54.658450,62.937425,56.896159,24.038763,15.668439,18.990386,43.046199,4.305750
2,3,0.113957,4.000000,0.477782,0.584625,0.470478,0.453323,0.464063,0.668704,0.457184,0.263226,45.233984,56.282563,53.228624,61.810772,49.068161,37.774549,27.885620,28.838040,5.495805
3,4,0.060326,2.791667,0.481941,0.548095,0.735996,0.358094,0.545610,0.687818,0.149768,0.221751,36.345609,59.752855,58.237522,48.133199,48.125187,36.216763,47.195610,71.683832,1.372225
4,5,0.074184,5.416667,0.491245,0.515532,0.506885,0.398667,0.391867,0.698720,0.416998,0.262095,49.431184,42.433133,51.137400,72.327924,40.661168,32.672766,30.081045,30.895217,2.435703
5,6,0.034797,3.041667,0.796917,0.793340,0.438915,0.414000,0.462889,0.480992,0.139698,0.127042,70.670117,50.309584,51.898738,35.528663,43.780559,36.801326,55.171784,82.502483,2.035048
6,7,0.044003,2.625000,0.708718,0.573122,0.427604,0.494779,0.480610,0.667772,0.365168,0.169456,47.655876,39.596612,49.351281,60.827172,50.565212,40.802447,29.124895,18.478613,3.999358
7,8,0.040644,2.416667,0.451987,0.648557,0.635433,0.480661,0.452204,0.583349,0.473851,0.369414,53.054848,54.512529,47.841975,51.532586,16.150636,47.266064,45.467078,22.006059,2.046468
8,9,0.050813,2.500000,0.434101,0.649425,0.519547,0.397346,0.619001,0.611399,0.341369,0.313512,47.584211,62.789917,52.798615,42.864737,37.867718,35.476285,33.956706,52.128299,2.015993
9,10,0.048532,2.625000,0.529480,0.627251,0.527712,0.329129,0.381600,0.746258,0.536557,0.476447,46.907588,53.255871,59.005440,68.562323,35.168216,33.362402,38.511443,48.576923,2.761363


In [127]:
Std_Original_Signature = pandas.DataFrame(Original_Signature_Table.groupby('Person').std().reset_index())

In [113]:
del Original_Signature_Table_1record['Category']

In [121]:
Original_Signature_Table_zscore = (Original_Signature_Table_1record - Original_Mean_1record)/ Std_Original_1record

In [122]:
Original_Signature_Table_zscore

,Person,Norm_area,connected_comps,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x,tan_topleft_y,tan_topright_x,tan_topright_y,tan_bottomleft_x,tan_bottomleft_y,tan_bottomright_x,tan_bottomright_y,Aspect_Ratio
0,0.0,0.754367,-0.908928,0.422228,-0.224749,1.920628,-0.334725,0.974219,1.538012,0.042991,-0.332599,-0.29432,-0.710005,-0.169733,0.840173,0.819288,-0.347877,0.056647,-0.816018,-1.162572


In [123]:
Original_Signature_Table_zscore == 0.0

,Person,Norm_area,connected_comps,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x,tan_topleft_y,tan_topright_x,tan_topright_y,tan_bottomleft_x,tan_bottomleft_y,tan_bottomright_x,tan_bottomright_y,Aspect_Ratio
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [120]:
Original_Signature_Table_zscore

0.0